# 何のnotebook?
- kaggleの練習用notebook
- いきなりコンペをみても、いったい何がなんだかわからないので、まずはわかりやすいデータセットをいじるところから始めたい
https://www.kaggle.com/c/mercari-price-suggestion-challenge/data?select=sample_submission_stg2.csv.zip

## 必要なこと
- どんな技術を知るべきなのかをまとめる
- 学んだ技術の振り返り

In [3]:
# データ読み込み
import pandas as pd
df = pd.read_table("2020-06-15_mercari_train.tsv")

In [4]:
# 今回はtrainデータで検定方法の振り返り
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [21]:
# 仮説：conditionによってpriceは変わらない。2と3は同じprice水準になる。
# データ作成
A_ = df.query('item_condition_id == 2')
B_ = df.query('item_condition_id == 3')
A = A_.price
B = B_.price

In [22]:
## pythonでt検定を行うなら(状況からStudent(スチューデント)のt検定を用いる)
import numpy as np
from scipy import stats
### 帰無仮説をA=Bとして、スチューデントのT検定を行う
stats.ttest_ind(A, B)

Ttest_indResult(statistic=11.173819982713205, pvalue=5.504264754353953e-29)

- p値が5％を下回っているので、帰無仮説は棄却。AとBで価格には有意な差がある

In [24]:
print(A.describe())

count    375479.000000
mean         27.563225
std          42.125698
min           0.000000
25%          11.000000
50%          17.000000
75%          29.000000
max        2004.000000
Name: price, dtype: float64


In [25]:
print(B.describe())

count    432161.000000
mean         26.540711
std          40.030681
min           0.000000
25%          11.000000
50%          16.000000
75%          28.000000
max        2000.000000
Name: price, dtype: float64


- つまりA>Bである。効果量は

In [26]:
# サンプル数の設計
from statsmodels.stats.power import tt_ind_solve_power
# 有意水準(一般的に使われている5%とする)
sig   = 0.05
# 検定力(一般的に使われている80％とする)
power = 0.8
# レシオ  サンプルサイズの比率。1:1の場合、以下で指定
ratio =375479/432161
# 片側検定(A>Bを示したいならlarger,逆ならsmaller)
alternative ='larger'
n = tt_ind_solve_power(nobs1=375479, alpha=sig, power=power, alternative=alternative, ratio=ratio)
# 全体のサンプル数算出
print("effect_size=effect_size: ", n)

effect_size=effect_size:  0.00595311890806683


- 効果量はめちゃくちゃちっちゃい(ほんとかな…？)